In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D
from sklearn.model_selection import train_test_split
print(tf.__version__)

c:\Users\Robby\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Robby\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\Robby\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


2.13.0


In [ ]:
pd.set_option('display.max_colwidth', None )
pd.set_option('display.max_rows', None)

In [ ]:
# Load data
train_df = pd.read_csv(r"C:\Users\Robby\Downloads\PROJECT\Toxic Comment Classification\train.csv")

In [ ]:
train_df.fillna(' ', inplace= True)

In [ ]:
import re
import string
def clean_punctuation(text):
    text = str(text)
    # Case folding
    text = text.lower()
    # Menghapus spasi berlebih
    text = ' '.join(text.split())
    # substitusi kata
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"r", "", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"'ll", " will", text)
    text = re.sub(r"'ve", " have", text)
    text = re.sub(r"'re", " are", text)
    text = re.sub(r"'d", " would", text)
    text = re.sub(r"'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    # Menghapus karakter khusus, tanda baca
    text = re.sub(r'[-.,+"&\'#@;:{}`+=~/!?()]', '', text)\
    # memastikan tanda baca terhapus
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub("(\W)"," ",text)
    # menghapus kata yang diapit oleh karakter s
    text = re.sub('S*dS*s*','', text)


    return text

In [ ]:
train_df['comment_text'] = train_df['comment_text'].apply(clean_punctuation)

In [ ]:
x = train_df['comment_text'].values

In [ ]:
y = train_df['toxic'].values

In [ ]:
max_features = 20000 # hanya mengambil 20000 kata yang paling sering muncul
max_text_length = 400 # panjang maksimum dari teks yang akan diolah max_features = 20000 # hanya mengambil 20000 kata yang paling sering muncul
max_text_length = 400 # panjang maksimum dari teks yang akan diolah

In [ ]:
x_tokenizer = text.Tokenizer(max_features) # membuat tokenizer, ini hanya berupa objek kosong yang akan menampung hasil tokenisasi

# hanya membangun sebuah kamus kata-kata yang ada dalam teks dan menghitung seberapa sering kata muncul, secara sederhana
# # hanya mengumpulkan statitik tentang frekeuensi kemunculan kata pada setiap teks
x_tokenizer.fit_on_texts(list(x))

# proses mengubah kata menjadi urutan token numerik berdasarkan kamus yang telah dibuat diatas
x_tokenized = x_tokenizer.texts_to_sequences(x)

In [ ]:
import pickle

# Menyimpan tokenizer ke dalam file pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(x_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# disebut juga dengan padding
# membuat sequence dengan panjang maksimum 400, dikarenakan dalam klasifikasi teks, model memerlukan input dengan panjang yang sama
x_train_val = sequence.pad_sequences(x_tokenized, maxlen=max_text_length)

In [ ]:
#!wget http://nlp.stanford.edu/data/glove.6B.zip
#!unzip -q glove.6B.zip

embedding_dim = 100
embedding_index = dict()
with open(r"C:\Users\Robby\Downloads\PROJECT\Toxic Comment Classification\glove.6B.100d.txt", encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0] # mengambil kata
        coef = np.asarray(values[1:], dtype='float32') # mengubah nilai koefisien menjadi float32
        embedding_index[word] = coef
print(f'Ditemukan {len(embedding_index)} vektor kata.')


Ditemukan 400000 vektor kata.


In [ ]:
embedding_matrix = np.zeros((max_features, embedding_dim))
for word, index in x_tokenizer.word_index.items():
    if index > max_features - 1:
        break
    else:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

### CNN MULTICHANNEL

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dropout, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense, concatenate
from tensorflow.keras.regularizers import l2


# Channel 1: Custom Embedding Matrix (Static)
input1 = Input(shape=(max_text_length,))
x1 = Embedding(max_features, embedding_dim,
               embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
               trainable=False)(input1)
x1 = Conv1D(250, 3, activation='relu')(x1)
x1 = MaxPooling1D()(x1)
x1 = GlobalMaxPooling1D()(x1)

# Channel 2: Trainable Embedding
input2 = Input(shape=(max_text_length,))
x2 = Embedding(max_features, embedding_dim, trainable=True)(input2)
x2 = Conv1D(250, 3, activation='relu')(x2)
x2 = MaxPooling1D()(x2)
x2 = GlobalMaxPooling1D()(x2)

# Menggabungkan kedua channel
merged = concatenate([x1, x2])

l2_reg = 0.001  # Regularisasi L2

# Tambahkan Dense layers dengan regularisasi
merged = Dense(250, activation='relu', kernel_regularizer=l2(l2_reg))(merged)
merged = Dropout(0.5)(merged)  # Tingkatkan dropout rate
output = Dense(1, activation='sigmoid')(merged)

# Membuat model
model = Model(inputs=[input1, input2], outputs=output)

model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 400)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 400)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 400, 100)     2000000     ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 400, 100)     2000000     ['input_2[0][0]']                
                                                                                              

In [ ]:
from sklearn.utils import class_weight
import numpy as np

# Menggunakan kolom 'toxic' sebagai label
labels = train_df['toxic'].values

# Menghitung bobot kelas
class_weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(labels),
    y=labels
)

# Mengonversi bobot kelas menjadi dictionary
class_weight_dict = {i : class_weights[i] for i in range(len(class_weights))}

In [ ]:
x_train , x_val, y_train, y_val = train_test_split(x_train_val, y, test_size= 0.15, random_state=1)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stopping = EarlyStopping(monitor='val_loss', patience=3)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.001)
callbacks_list = [early_stopping, reduce_lr]


In [ ]:
# Jika hanya memiliki satu representasi data
x_train_1, x_val_1, y_train, y_val = train_test_split(x_train_val, y, test_size=0.15, random_state=1)
x_train_2, x_val_2 = x_train_1, x_val_1  # Menggunakan data yang sama untuk kedua channel

In [ ]:
# Kompilasi model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy', 'Precision', 'Recall', 'AUC'])

# Sekarang, lakukan fitting model
history = model.fit(
    [x_train, x_train], y_train,
    batch_size=32,
    epochs=5,
    validation_data=([x_val, x_val], y_val),
    class_weight=class_weight_dict,
    callbacks=callbacks_list
)

Epoch 1/5
4239/4239 [==============================] - 824s 194ms/step - loss: 0.3094 - accuracy: 0.8967 - precision: 0.4785 - recall: 0.8767 - auc: 0.9557 - val_loss: 0.2931 - val_accuracy: 0.8854 - val_precision: 0.4541 - val_recall: 0.9328 - val_auc: 0.9708 - lr: 0.0010
Epoch 2/5
4239/4239 [==============================] - 981s 232ms/step - loss: 0.1966 - accuracy: 0.9275 - precision: 0.5749 - recall: 0.9335 - auc: 0.9795 - val_loss: 0.2994 - val_accuracy: 0.8895 - val_precision: 0.4637 - val_recall: 0.9332 - val_auc: 0.9687 - lr: 0.0010
Epoch 3/5
4239/4239 [==============================] - 931s 220ms/step - loss: 0.1529 - accuracy: 0.9423 - precision: 0.6311 - recall: 0.9566 - auc: 0.9875 - val_loss: 0.2183 - val_accuracy: 0.9157 - val_precision: 0.5369 - val_recall: 0.9085 - val_auc: 0.9688 - lr: 0.0010
Epoch 4/5
4239/4239 [==============================] - 876s 207ms/step - loss: 0.1224 - accuracy: 0.9553 - precision: 0.6892 - recall: 0.9718 - auc: 0.9917 - val_loss: 0.1866 - v

In [ ]:
# Melihat Akurasi
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

print("Akurasi Pelatihan:", train_acc)
print("Akurasi Validasi:", val_acc)

Akurasi Pelatihan: [0.8967006802558899, 0.9275408387184143, 0.9423084259033203, 0.9553360342979431, 0.9659306406974792]
Akurasi Validasi: [0.8854445219039917, 0.8895387649536133, 0.9156500697135925, 0.9341577291488647, 0.938168466091156]


In [ ]:
# 1. Memuat Data Test
test_df = pd.read_csv(r"C:\Users\Robby\Downloads\PROJECT\Toxic Comment Classification\test.csv")

# 2. Membersihkan Data Test
test_df.fillna(' ', inplace=True)
test_df['comment_text'] = test_df['comment_text'].apply(clean_punctuation)

# 3. Tokenisasi Data Test
x_test = test_df['comment_text'].values
x_test_tokenized = x_tokenizer.texts_to_sequences(x_test)

# 4. Padding Data Test
x_test_padded = sequence.pad_sequences(x_test_tokenized, maxlen=max_text_length)

# 5. Membuat Prediksi
predictions = model.predict([x_test_padded, x_test_padded])

4787/4787 [==============================] - 254s 53ms/step


In [ ]:
model.save(r'C:\Users\Robby\Downloads\PROJECT\Toxic Comment Classification\modelku.h5')  # menyimpan model Keras ke file .h5